In [ ]:
# Install kagglehub (only once)
!pip install kagglehub


In [ ]:
import kagglehub # Download latest version
path = kagglehub.dataset_download("mohamedasak/chest-x-ray-6-classes-dataset")#206.95mb
print("Path to dataset files:", path)


100%|██████████| 194M/194M [00:01<00:00, 185MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mohamedasak/chest-x-ray-6-classes-dataset/versions/1


In [ ]:
import os

# Check main folder
print(os.listdir(path))


['chest-xray']


In [ ]:
data_path = path + "/chest-xray"

print(os.listdir(data_path))


['val', 'test', 'train']


In [ ]:
print(os.listdir(data_path + "/train"))


['Emphysema', 'Pneumonia-Bacterial', 'Covid-19', 'Normal', 'Tuberculosis', 'Pneumonia-Viral']


In [ ]:
import tensorflow as tf
import numpy as np
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_path + "/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)




Found 14551 files belonging to 6 classes.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_path + "/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 1748 files belonging to 6 classes.


In [ ]:
class_names = train_ds.class_names
print("Diseases:", class_names)


Diseases: ['Covid-19', 'Emphysema', 'Normal', 'Pneumonia-Bacterial', 'Pneumonia-Viral', 'Tuberculosis']


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,734 (42.61 MB)

 Trainable params: 11,169,734 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

Epoch 1/2
455/455 ━━━━━━━━━━━━━━━━━━━━ 1798s 4s/step - accuracy: 0.6042 - loss: 1.0226 - val_accuracy: 0.7323 - val_loss: 0.6422
Epoch 2/2
455/455 ━━━━━━━━━━━━━━━━━━━━ 1736s 4s/step - accuracy: 0.7995 - loss: 0.4940 - val_accuracy: 0.8038 - val_loss: 0.4675


In [ ]:
model.save("lung_disease_model.keras")

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving abcd.jpg to abcd.jpg


In [ ]:
from tensorflow.keras.preprocessing import image

img_path = list(uploaded.keys())[0]

img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
predicted_disease = class_names[np.argmax(prediction)]

print("Predicted Disease:", predicted_disease)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Predicted Disease: Covid-19
